In [1]:
import pandas as pd
import os
import datetime
import json
from functools import reduce

In [2]:
ROOT_DIR = r"C:\dev\backend\tasks\samples\cmots"
asof_date = datetime.date(2023, 1, 21)
DIR_PATH = os.path.join(ROOT_DIR, asof_date.strftime("%d%m%Y"))

# Files needed to be refreshed from CMOTS ftp : amfimaster.csv, mfcat.csv, mfisin_all.csv, mutfund.csv

DIR_PATH

'C:\\dev\\backend\\tasks\\samples\\cmots\\21012023'

In [3]:
def get_df(dir_path, file_name, asof_date, lst_col, lst_drop_col):
    file_name = os.path.join(dir_path, file_name)
    print(file_name)
    if os.path.isfile(file_name):
        print('Reading the file into df...')
        df = pd.read_csv(file_name, sep ='|', names=lst_col, encoding= 'unicode_escape' )
        df.iloc[:, -1] = df.iloc[:, -1].str.split('<</EOR>>', expand=True)
        # print(df.head())
        df = df.drop(lst_drop_col, axis=1)
    else:
        df = pd.DataFrame()
    return df

In [4]:
# get data for amfi scheme master
# 1] amfimaster.csv
lst_col = ["MF_COCODE","MF_SCHCODE","SRNO","SCH_NAME","AMFICODE","FLAG"]
lst_used = ["MF_COCODE","MF_SCHCODE","SCH_NAME","AMFICODE"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_amfimas = get_df(DIR_PATH, "amfimaster.csv", asof_date, lst_col, lst_drop)
df_amfimas.sort_values("MF_SCHCODE", inplace=True)
df_amfimas.head()

C:\dev\backend\tasks\samples\cmots\21012023\amfimaster.csv
Reading the file into df...


,MF_COCODE,MF_SCHCODE,SCH_NAME,AMFICODE
104,5946,1,Aditya Birla SL Equity Hybrid '95 Fund (G),103155
105,5946,2,Aditya Birla SL Equity Hybrid '95 Fund (IDCW),103154
106,5946,7,Aditya Birla SL Tax Relief '96 (IDCW),103164
108,5946,8,Aditya Birla SL Low Duration Fund (G),103192
109,5946,9,Aditya Birla SL Flexi Cap Fund (G),103166


In [5]:
# 2] mfcat.csv
lst_col = ["MF_COCODE","GRPCODE","MF_SCHCODE","GRPNAME","MF_SCHNAME",
           "PLANNAME","MPLANNAME","OPTIONNAME","MOPTIONNAME"]
lst_used = ["MF_SCHCODE","GRPCODE","GRPNAME",
            "PLANNAME","OPTIONNAME"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_mfcat = get_df(DIR_PATH, "mfcat.csv", asof_date, lst_col, lst_drop)
df_mfcat.sort_values("MF_SCHCODE", inplace=True)
df_mfcat.head()

C:\dev\backend\tasks\samples\cmots\21012023\mfcat.csv
Reading the file into df...


,GRPCODE,MF_SCHCODE,GRPNAME,PLANNAME,OPTIONNAME
0,ABSL1,1,Aditya Birla SL Equity Hybrid '95 Fund,Regular,Growth
1,ABSL1,2,Aditya Birla SL Equity Hybrid '95 Fund,Regular,Dividend
2,ABSL7324,7,Aditya Birla SL Tax Relief '96,Regular,Dividend
3,ABSL8,8,Aditya Birla SL Low Duration Fund,Regular,Growth
4,ABSL9,9,Aditya Birla SL Flexi Cap Fund,Regular,Growth


In [6]:
# 3] mfisin_all_v2.csv
lst_col = ["MF_SCHCODE","ISIN"]

df_mfisin = get_df(DIR_PATH, "mfisin_all.csv", asof_date, lst_col, [])
#df_mfisin.sort_values("MF_SCHCODE", inplace=True)
df_mfisin.head()

C:\dev\backend\tasks\samples\cmots\21012023\mfisin_all.csv
Reading the file into df...


,MF_SCHCODE,ISIN
0,1,INF209K01BT5
1,2,INF209K01BS7
2,2,INF209K01EE1
3,7,INF209K01090
4,7,INF209K01CB1


In [7]:
df_final = pd.merge(df_amfimas, df_mfcat, on="MF_SCHCODE")
df_final = pd.merge(df_final, df_mfisin, on="MF_SCHCODE")
df_final['SCH_NAME'] = df_final['SCH_NAME'].str.replace("'","")
df_final['GRPNAME'] = df_final['GRPNAME'].str.replace("'","")
df_final.head()
df_final.to_excel(r'C:\dev\cmots\sql\02a_CMOTS_Plan_Data_Insert_Script.xlsx')

In [8]:
df_final2 = pd.merge(df_amfimas, df_mfcat, on="MF_SCHCODE")
df_final2['SCH_NAME'] = df_final2['SCH_NAME'].str.replace("'","")
df_final2['GRPNAME'] = df_final2['GRPNAME'].str.replace("'","")
df_final2.head()
df_final2.to_excel(r'C:\dev\cmots\sql\02b_CMOTS_Fund_Data_Insert_Script.xlsx')

In [9]:
# 3] mutfund.csv
lst_col = ["MF_COCODE","AMC_NAME","FUND_TYPE","INCORPORAT","AMC",
           "ADD_1","ADD_2","ADD_3","CITY","PIN_CODE","TEL_NO_1",
           "TEL_NO_2","FAX_NO","WEBSITE_UR","E_MAIL"]
lst_used = ["MF_COCODE","AMC_NAME","FUND_TYPE","AMC"]
lst_drop = [item for item in lst_col if item not in lst_used]

df_mutfund = get_df(DIR_PATH, "mutfund.csv", asof_date, lst_col, lst_drop)
df_mutfund.sort_values("MF_COCODE", inplace=True)
# df_mutfund.head()
# df_mutfund.to_excel(r'C:\dev\backend\tasks\samples\cmots\27122022\CMOTS_AMC_Mapping_Master_27122022.xlsx')

C:\dev\backend\tasks\samples\cmots\21012023\mutfund.csv
Reading the file into df...
